In [1]:
import numpy as np
import numpy.random
from numpy.fft import fft2, ifft2
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

# Nonlocal Full Module Test with 2D reaction diffusion equation

In [2]:
# Define the reaction-diffusion PDE in the Fourier (kx, ky) space
def reaction_diffusion(t, uvt, K22, d1, d2, beta, n, N):
    ut = np.reshape(uvt[:N], (n, n))
    vt = np.reshape(uvt[N : 2 * N], (n, n))
    u = np.real(ifft2(ut))
    v = np.real(ifft2(vt))
    u3 = u ** 3
    v3 = v ** 3
    u2v = (u ** 2) * v
    uv2 = u * (v ** 2)
    utrhs = np.reshape((fft2(u - u3 - uv2 + beta * u2v + beta * v3)), (N, 1))
    vtrhs = np.reshape((fft2(v - u2v - v3 - beta * u3 - beta * uv2)), (N, 1))
    uvt_reshaped = np.reshape(uvt, (len(uvt), 1))
    uvt_updated = np.squeeze(
        np.vstack(
            (-d1 * K22 * uvt_reshaped[:N] + utrhs, 
             -d2 * K22 * uvt_reshaped[N:] + vtrhs)
        )
    )
    return uvt_updated

In [3]:
# Generate the data
t = np.linspace(0, 10, int(10 / 0.1))
d1 = 0.1
d2 = 0.1
beta = 1.0
L = 20  # Domain size in X and Y directions
n = 64  # Number of spatial points in each direction
N = n * n
x_uniform = np.linspace(-L / 2, L / 2, n + 1)
x = x_uniform[:n]
y = x_uniform[:n]
n2 = int(n / 2)
# Define Fourier wavevectors (kx, ky)
kx = (2 * np.pi / L) * np.hstack((np.linspace(0, n2 - 1, n2), 
                                  np.linspace(-n2, -1, n2)))
ky = kx
# Get 2D meshes in (x, y) and (kx, ky)
X, Y = np.meshgrid(x, y)
KX, KY = np.meshgrid(kx, ky)
K2 = KX ** 2 + KY ** 2
K22 = np.reshape(K2, (N, 1))

m = 1  # number of spirals

# define our solution vectors
u = np.zeros((len(x), len(y), len(t)))
v = np.zeros((len(x), len(y), len(t)))

# Initial conditions
u[:, :, 0] = np.tanh(np.sqrt(X ** 2 + Y ** 2)) * np.cos(
    m * np.angle(X + 1j * Y) - (np.sqrt(X ** 2 + Y ** 2))
)
v[:, :, 0] = np.tanh(np.sqrt(X ** 2 + Y ** 2)) * np.sin(
    m * np.angle(X + 1j * Y) - (np.sqrt(X ** 2 + Y ** 2))
)

# uvt is the solution vector in Fourier space, so below
# we are initializing the 2D FFT of the initial condition, uvt0
uvt0 = np.squeeze(
    np.hstack(
        (np.reshape(fft2(u[:, :, 0]), (1, N)), 
         np.reshape(fft2(v[:, :, 0]), (1, N)))
    )
)

# Solve the PDE in the Fourier space, where it reduces to system of ODEs
uvsol = solve_ivp(
    reaction_diffusion, 
    (t[0], t[-1]), 
    y0=uvt0, 
    t_eval=t, 
    args=(K22, d1, d2, beta, n, N)
)
uvsol = uvsol.y

# Reshape things and ifft back into (x, y, t) space from (kx, ky, t) space
for j in range(len(t)):
    ut = np.reshape(uvsol[:N, j], (n, n))
    vt = np.reshape(uvsol[N:, j], (n, n))
    u[:, :, j] = np.real(ifft2(ut))
    v[:, :, j] = np.real(ifft2(vt))

dt = t[1] - t[0]
dx = x[1] - x[0]
dy = y[1] - y[0]

u_sol = u
v_sol = v

U = np.zeros((n, n, len(t), 2))
U[:, :, :, 0] = u_sol
U[:, :, :, 1] = v_sol
X, Y, T = np.meshgrid(x, y, t, indexing='ij')
XYT = np.transpose([X, Y, T], [1, 2, 3, 0])

In [4]:
def setup_subdomains(spatial_grid, K):
    """Setup subdomains for nonlocal computation

        Output format: A list of ndim elements containing lists of bounds of subdomains. In a 2D
        case, an example would be [[[0, 1], [2, 4], [5, 9]], [[0, 4], [5, 12]]].
    """
#       spatial grid is this 2D tensor that stores the grid. 
#       We split each dimension into K (mostly) equally sized subdomains, where K is a parameter. 
#       There will be a total of K^ndim subdomains 
    bounds = []
#     -1 to adjust for the last dimension, which indicates feature.
    for i in np.arange(np.array(spatial_grid).ndim-1):
#         does spatio-temporal grid have to have the same number of points in each spatial dimension?
        length = np.shape(spatial_grid)[1]
    
        if length//K[i] < 2:
#             replace this with warning or break point
            print("Warning: too many subdomains created in axis", i)
        
        subdomain_length = length//K[i]
        remain = length % K[i]
        size = np.zeros(K[i]) + subdomain_length
        size[:remain] += 1
        bound = np.cumsum(size)
        bound = [int(s) for s in bound]
        bounds.append(bound)
    return bounds

'''A generalization from https://stackoverflow.com/questions/29142417/4d-position-from-1d-index'''
def nd_iterator(index, K):
    '''
    index: the 1D index of the nd item to recover
    K: number of items per dimension, corresponding to each of the n dimensions.
    
    return:
    nd index of the item with 1D index to be "index"
    '''
    nd_index = [index % K[0]]
    dividor = 1
    remaining_index = index
    for i in np.arange(len(K)-1):
        remaining_index -= nd_index[i]*dividor
        dividor *= K[i]
        this_index = remaining_index // dividor % K[i+1]
        nd_index.append(this_index)
    return nd_index

def subdomain_iterator(index, bounds, K):
    '''
    bounds: subdomain dividor, in standard setup_subdomains format
    nd_index: nd index of a specific subdomain, in nd_iterator standard output format.
    
    return:
    boundary of that subdomain in standard nonlocal format.
    '''
    nd_index = nd_iterator(index, K)
    bound = []
    for i in np.arange(len(nd_index)):
        if nd_index[i] == 0:
            bound.append([0, bounds[i][0]-1])
        else:
            bound.append([bounds[i][nd_index[i]-1], bounds[i][nd_index[i]]-1])
    
    return np.reshape(bound, (len(nd_index), 2))

def spatial_iterator(index, spatiotemporal_grid, K):
    '''
    bounds: subdomain dividor, in standard setup_subdomains format
    nd_index: nd index of a specific subdomain, in nd_iterator standard output format.
    
    return:
    nd value of x at that index.
    '''
    nd_index = nd_iterator(index, K)
#     append time point
    nd_index.append(0)
    
#     compute slicing index for the last dimension
    num_dim = len(np.shape(spatiotemporal_grid))-1
#     append last axis
    nd_index.append(slice(0, num_dim-1, 1))
    
    return spatiotemporal_grid[tuple(nd_index)]

#indicator function has been modified
# modify it so it can kicks the points with not enough dimension to be zero
def indicator(x_points, endpts):
    '''
    if x value is inside the bound, return 1. Otherwise, return 0

    Require:
        endpts: (left_bound, right_bound).
        x, left_bound, right_bound must have the same dimension

    Parameters: 

            x: m x (n+1) vector representing the index of point to check (Time dimension should be excluded)

            endpts: 2d (n x 2) array of index. First dimension is all the spatial dimensions, and second dimension are 
                    left and right bound of the subdomain in terms of index

    `return: 
            1 x n matrix that consist of 1 and 0's indicating whether the point is inside the subdomain or not
    '''
    x = np.copy(x_points)
    x = x.T
    
    # iterate through each dimensions
    for i in np.arange(np.shape(endpts)[0]):
        upper_limit = endpts[i][1]
        lower_limit = endpts[i][0]
        x[i] = np.multiply(np.abs(x[i] - 0.5*(upper_limit+lower_limit))<=0.5*(upper_limit-lower_limit), 1)
        
    # Transpose back to original shape
    x = x.T
    return np.add(np.sign(np.subtract(np.sum(x, axis=1), len(endpts))), 1)

def get_1D_weight(grid, endpt):
    '''
    Parameters: 
        grid: an 1D array that contains the value of the corresponding dimension of each grid points.

        endpts: 1 x 2 array 
            the first element is the left endpoints of this dimensions in terms of index,
            second element is the left endpoints of this dimensions in terms of index,
    '''

    if endpt[0] >= endpt[1]:
        raise ValueError("Illegal Endpoints.")

#     initialize a bunch of 0,
    weight = np.zeros(endpt[1]-endpt[0])

#     find the index at which we enter Omega_k in this axis
    start = endpt[0]
    end = endpt[1]

#     start and end index has different equation for weight, so we do those first
    weight[0] = 1/2*(grid[start+1]-grid[start])
    weight[-1] = 1/2*(grid[end]-grid[end-1])
    weight[1:-1] = np.array([0.5 * (grid[(start+2):(end)] - grid[start:(end-2)])])

    return weight

def get_full_weight(weights):
    '''
    weights: a list of lists, where each inner list is the 1D weight in a dimension. 
    '''
    ndim = len(weights)
    W_F = np.array(weights[0])
    for w in np.arange(ndim-1)+1:
        index = [slice(None)]*(w+1)
        index[-1] = np.newaxis
        W_F = W_F[tuple(index)] * np.array(weights[w])

    return W_F

# Methods to filter data matrix X
def retrieve_data_mat(spatiotemporal_grid, X):
    overallShape = list(np.shape(spatiotemporal_grid)[:-1]) + [np.shape(X)[-1]]
    return X.reshape(overallShape)

def filterX(X, j, bound, t_ind):
#     filter by feature j first
    index = [0]*len(np.shape(X))
    for i in range(np.shape(bound)[0]):
        index[i] = slice(bound[i][0], bound[i][1])
    index[-2] = t_ind
    index[-1] = j
    return X[tuple(index)]

def get_theta_nonloc(X, spatiotemporal_grid, j, k, kprime, bounds, K):
    '''
    Parameters:
        spatiotemporal_grid: The spatiotemporal_grid that contains information about spatial and time points.
        j: the index of u that we are looking for
        k: the index of subdomain to be used by the indicator function
        kprime: the index of the subdomain to be used as boundary of integral
        bounds: boundary of each subdomain correspond to each dimension in terms of indexing. 

    return: 
        vector Theta^nonloc_p
    '''
    spatio_grid = spatiotemporal_grid[..., 0, :-1]
#     get number and space and time points
    num_t = np.shape(spatiotemporal_grid)[-2]
    num_x = np.prod(np.shape(spatiotemporal_grid)[:-2])
    
    
#     Since all the spatiotemporal_grid contains indication, time and spatial dimensions, and there must be 1 time dimension
#     the number of spatial is then given as following
    grid_ndim = len(np.shape(spatiotemporal_grid))-2
    
#     construct shape of x
    x_shape = np.shape(spatiotemporal_grid)[:-2]
    
    
    integral = np.zeros((num_x, num_t))
    
#     get coeff
    coeff = np.zeros((num_x, num_t))
    coeff_bounds = subdomain_iterator(k, bounds, K)
    x_flat = np.zeros((2, 4096))
    x_flat[0, :] = np.reshape(spatio_grid[:, :, 0], 4096)
    x_flat[1, :] = np.reshape(spatio_grid[:, :, 1], 4096)
    
    coeff = indicator(x_flat.T, coeff_bounds)

#    get integral
    integral_bounds = subdomain_iterator(kprime, bounds, K)
    for t in np.arange(num_t):
        # find weights
        # All the 1D weights will be stored in a 2D matrix as cols
        weights = []
        for i in np.arange(grid_ndim):
            # +2 to account for the time and indication dimension
            index = [0]*(grid_ndim+2)
            # Time is always the second to last dimension, which is filtered here
            index[-2] = t
            index[i] = slice(None)
            index[-1] = i
    
    #         we now get the 1D grid by filtering by the index created
            this_dim = spatiotemporal_grid[tuple(index)]
            weight = get_1D_weight(this_dim, integral_bounds[i])
            weights.append(weight)

        W_F = get_full_weight(weights)
        F = filterX(X, j, integral_bounds, t)

        integral[:, t] = np.sum(np.multiply(W_F, F))
    np.shape(integral)
        
#     final product
    theta_nonloc_p = (coeff * integral.T).T

    return theta_nonloc_p

def sample_test_space(spatiotemporal_grid, x):
    spatial_grid = XYT[..., 0, :-1]
    dims = spatial_grid.shape[:-1]
    grid_ndim = len(dims)
    
#     number of space points sampled
    n_samples_full = np.prod(np.shape(x)[:grid_ndim])
#     number of features at each space point
    n_features = np.shape(x)[-1]-1

    K = [2]*grid_ndim
    subdomain_bounds = setup_subdomains(spatial_grid, K)
    
    res = []
    
    tot_iter = n_features * np.prod(K) * np.prod(K)

    for j in np.arange(n_features):
        for k in np.arange(np.prod(K)):
            for kprime in np.arange(np.prod(K)):
                res.append(get_theta_nonloc(x, spatiotemporal_grid, j, k, kprime, subdomain_bounds, K))
    return res

# Data matrix is U, spatiotemporal_grid is XYT

In [5]:
a = sample_test_space(XYT, U)

[2, 2]
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)
(4096,)


In [6]:
np.shape(a)

(16, 4096, 100)

In [7]:
np.shape(U)

(64, 64, 100, 2)

In [8]:
np.shape(XYT)

(64, 64, 100, 3)